# Energy vs Solar Radiation Correlation Analysis
## Kiểm tra sản lượng năng lượng vs bức xạ mặt trời

Phân tích mối quan hệ giữa sản lượng năng lượng (energy) và bức xạ mặt trời (shortwave_radiation) cho 5 trạm PV.

**Kỳ vọng**: Phải có mối quan hệ đồng biến mạnh (positive correlation) - khi bức xạ tăng thì sản lượng phải tăng.

**Dữ liệu**: 2025-06-01 đến 2025-11-08 (~ 161 ngày, ~19K bản ghi trên mỗi trạm)

**Mục tiêu**:
1. Load dữ liệu từ Silver layer (toàn bộ khoảng thời gian)
2. Kiểm tra correlation Pearson giữa energy và radiation
3. Tìm anomalies (dữ liệu bất thường)
4. Visualize scatter plots và time series

## 1. Import Required Libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

print("✓ Imports successful")

✓ Imports successful


## 2. Load Facility Data and Create Trino Connection

In [13]:
# Install trino-python-client if needed
import subprocess
import sys

try:
    import trino
except ImportError:
    print("Installing trino-python-client...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "trino", "-q"])
    import trino

from trino.dbapi import connect

# Create connection to Trino (port 8081, not 8080)
conn = connect(
    host='localhost',
    port=8081,
    user='trino',
    catalog='iceberg',
    schema='silver'
)

print("✓ Trino connection established")

✓ Trino connection established


## 3. Load Energy and Weather Data from Silver Layer

In [14]:
# Load energy data (power output)
query_energy = """
SELECT 
    date_hour,
    facility_code,
    facility_name,
    network_region,
    energy_mwh,
    intervals_count,
    is_valid
FROM iceberg.silver.clean_hourly_energy
ORDER BY facility_code, date_hour
"""

energy_df = pd.read_sql(query_energy, conn)
energy_df['date_hour'] = pd.to_datetime(energy_df['date_hour'])

print(f"✓ Loaded {len(energy_df)} energy records")
print(f"  Date range: {energy_df['date_hour'].min()} to {energy_df['date_hour'].max()}")
print(f"  Facilities: {energy_df['facility_code'].unique().tolist()}")

✓ Loaded 19315 energy records
  Date range: 2025-06-01 00:00:00+00:00 to 2025-11-08 23:00:00+00:00
  Facilities: ['BNGSF1', 'CLARESF', 'COLEASF', 'GANNSF', 'NYNGAN']


In [15]:
# Load weather data (solar radiation)
query_weather = """
SELECT 
    date_hour,
    facility_code,
    facility_name,
    shortwave_radiation,
    direct_radiation,
    diffuse_radiation,
    direct_normal_irradiance,
    is_valid
FROM iceberg.silver.clean_hourly_weather
ORDER BY facility_code, date_hour
"""

weather_df = pd.read_sql(query_weather, conn)
weather_df['date_hour'] = pd.to_datetime(weather_df['date_hour'])

print(f"✓ Loaded {len(weather_df)} weather records")
print(f"  Date range: {weather_df['date_hour'].min()} to {weather_df['date_hour'].max()}")
print(f"  Facilities: {weather_df['facility_code'].unique().tolist()}")

✓ Loaded 19320 weather records
  Date range: 2025-06-01 00:00:00+00:00 to 2025-11-08 23:00:00+00:00
  Facilities: ['BNGSF1', 'CLARESF', 'COLEASF', 'GANNSF', 'NYNGAN']


## 4. Merge Energy and Weather Data

In [16]:
# Merge energy and weather on facility_code and date_hour
df = pd.merge(
    energy_df,
    weather_df,
    on=['date_hour', 'facility_code'],
    how='inner',
    suffixes=('_energy', '_weather')
)

print(f"✓ Merged data: {len(df)} records")
print(f"\nData shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

✓ Merged data: 19315 records

Data shape: (19315, 13)

Columns: ['date_hour', 'facility_code', 'facility_name_energy', 'network_region', 'energy_mwh', 'intervals_count', 'is_valid_energy', 'facility_name_weather', 'shortwave_radiation', 'direct_radiation', 'diffuse_radiation', 'direct_normal_irradiance', 'is_valid_weather']


## 5. Data Exploration

In [17]:
print("="*80)
print("DATA SUMMARY BY FACILITY")
print("="*80)

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    print(f"\n{facility} ({fac_data['facility_name_energy'].iloc[0]}) - Region: {fac_data['network_region'].iloc[0]}")
    print(f"  Records: {len(fac_data)}")
    print(f"  Energy (MWh): mean={fac_data['energy_mwh'].mean():.4f}, std={fac_data['energy_mwh'].std():.4f}")
    print(f"  Radiation (W/m²): mean={fac_data['shortwave_radiation'].mean():.2f}, std={fac_data['shortwave_radiation'].std():.2f}")
    print(f"  Missing energy: {fac_data['energy_mwh'].isna().sum()}")
    print(f"  Missing radiation: {fac_data['shortwave_radiation'].isna().sum()}")

DATA SUMMARY BY FACILITY

BNGSF1 (Bungala One) - Region: SA1
  Records: 3863
  Energy (MWh): mean=17.1818, std=27.8889
  Radiation (W/m²): mean=174.56, std=255.07
  Missing energy: 0
  Missing radiation: 0

CLARESF (Clare) - Region: QLD1
  Records: 3863
  Energy (MWh): mean=14.2325, std=24.7841
  Radiation (W/m²): mean=220.82, std=300.72
  Missing energy: 0
  Missing radiation: 0

COLEASF (Coleambally) - Region: NSW1
  Records: 3863
  Energy (MWh): mean=25.3839, std=40.1454
  Radiation (W/m²): mean=167.94, std=248.91
  Missing energy: 0
  Missing radiation: 0

GANNSF (Gannawarra) - Region: VIC1
  Records: 3863
  Energy (MWh): mean=9.3603, std=13.7058
  Radiation (W/m²): mean=158.31, std=235.51
  Missing energy: 0
  Missing radiation: 0

NYNGAN (Nyngan) - Region: NSW1
  Records: 3863
  Energy (MWh): mean=14.9758, std=25.9754
  Radiation (W/m²): mean=186.96, std=271.15
  Missing energy: 0
  Missing radiation: 0


## 6. Calculate Correlation Between Energy and Radiation

In [18]:
print("="*80)
print("PEARSON CORRELATION: Energy vs Shortwave Radiation")
print("="*80)
print("Expected: Strong positive correlation (r > 0.7)")
print()

correlation_results = []

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    
    # Remove NaN values for correlation calculation
    clean_data = fac_data[['energy_mwh', 'shortwave_radiation']].dropna()
    
    if len(clean_data) > 1:
        pearson_r, p_value = stats.pearsonr(clean_data['energy_mwh'], clean_data['shortwave_radiation'])
        correlation_results.append({
            'facility_code': facility,
            'facility_name': fac_data['facility_name_energy'].iloc[0],
            'region': fac_data['network_region'].iloc[0],
            'pearson_r': pearson_r,
            'p_value': p_value,
            'significance': 'YES' if p_value < 0.05 else 'NO'
        })
        
        status = '✓' if pearson_r > 0.7 else '⚠' if pearson_r > 0.5 else '✗'
        print(f"{status} {facility}: r={pearson_r:.4f} (p={p_value:.2e}) - {'STRONG' if pearson_r > 0.7 else 'MODERATE' if pearson_r > 0.5 else 'WEAK'} correlation")

corr_df = pd.DataFrame(correlation_results)
print(f"\n{corr_df.to_string(index=False)}")

PEARSON CORRELATION: Energy vs Shortwave Radiation
Expected: Strong positive correlation (r > 0.7)

⚠ BNGSF1: r=0.5592 (p=1.06e-316) - MODERATE correlation
✗ CLARESF: r=0.4815 (p=2.08e-223) - WEAK correlation
✓ COLEASF: r=0.7668 (p=0.00e+00) - STRONG correlation
✓ GANNSF: r=0.8471 (p=0.00e+00) - STRONG correlation
⚠ NYNGAN: r=0.6405 (p=0.00e+00) - MODERATE correlation

facility_code facility_name region  pearson_r  p_value significance
       BNGSF1   Bungala One    SA1     0.5592   0.0000          YES
      CLARESF         Clare   QLD1     0.4815   0.0000          YES
      COLEASF   Coleambally   NSW1     0.7668   0.0000          YES
       GANNSF    Gannawarra   VIC1     0.8471   0.0000          YES
       NYNGAN        Nyngan   NSW1     0.6405   0.0000          YES


## 7. Detect Anomalies - Hours with Mismatch

In [19]:
print("="*80)
print("ANOMALY DETECTION: Energy-Radiation Mismatch")
print("="*80)
print("\nLooking for hours where:")
print("  - High radiation but LOW energy (underperformance)")
print("  - Low radiation but HIGH energy (impossible)")
print()

# Create quartiles for each facility (handling NaN properly)
def safe_qcut(x):
    try:
        result = pd.qcut(x.dropna(), q=4, labels=['Q1_low', 'Q2', 'Q3', 'Q4_high'], duplicates='drop')
        return pd.Series(result, index=x.index)
    except:
        return pd.Series([None] * len(x), index=x.index)

df['radiation_quartile'] = df.groupby('facility_code')['shortwave_radiation'].transform(safe_qcut)
df['energy_quartile'] = df.groupby('facility_code')['energy_mwh'].transform(safe_qcut)

# Find mismatches
anomalies = df[
    ((df['radiation_quartile'] == 'Q4_high') & (df['energy_quartile'] == 'Q1_low')) |  # High rad, low energy
    ((df['radiation_quartile'] == 'Q1_low') & (df['energy_quartile'] == 'Q4_high'))   # Low rad, high energy
]

print(f"\nTotal anomalies found: {len(anomalies)} / {len(df)} ({100*len(anomalies)/len(df):.2f}%)")

if len(anomalies) > 0:
    print(f"\nTop 10 anomalies:")
    anomalies_sorted = anomalies.sort_values('date_hour')
    print(anomalies_sorted[['date_hour', 'facility_code', 'energy_mwh', 'shortwave_radiation', 'radiation_quartile', 'energy_quartile']].head(10).to_string())

ANOMALY DETECTION: Energy-Radiation Mismatch

Looking for hours where:
  - High radiation but LOW energy (underperformance)
  - Low radiation but HIGH energy (impossible)


Total anomalies found: 0 / 19315 (0.00%)


## 8. Scatter Plot: Energy vs Radiation by Facility

In [20]:
# Create scatter plot
fig = px.scatter(
    df,
    x='shortwave_radiation',
    y='energy_mwh',
    color='facility_code',
    hover_data=['facility_code', 'date_hour', 'intervals_count'],
    title='Energy Production vs Solar Radiation by Facility',
    labels={
        'shortwave_radiation': 'Shortwave Radiation (W/m²)',
        'energy_mwh': 'Energy (MWh)'
    },
    height=600,
    width=1000
)

fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.show()

## 9. Time Series: Energy and Radiation Over Time

In [21]:
# Create time series plot for 1 month of data for each facility
facilities = sorted(df['facility_code'].unique())
n_facilities = len(facilities)

# Select 1 month of data (first complete month in dataset)
start_date = df['date_hour'].min()
end_date = start_date + pd.Timedelta(days=30)  # ~1 month

df_month = df[(df['date_hour'] >= start_date) & (df['date_hour'] < end_date)]

print(f"📊 Plotting data from {start_date.date()} to {end_date.date()} (~1 month)")
print(f"   Total records: {len(df_month):,}")

fig = make_subplots(
    rows=n_facilities, cols=1,
    subplot_titles=[f"{fac}" for fac in facilities],
    specs=[[{"secondary_y": True}]] * n_facilities,
    vertical_spacing=0.08
)

for idx, facility in enumerate(facilities, 1):
    fac_data = df_month[df_month['facility_code'] == facility].sort_values('date_hour')
    
    # Add energy line (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=fac_data['date_hour'],
            y=fac_data['energy_mwh'],
            name=f'{facility} Energy',
            line=dict(color='blue', width=2),
            legendgroup=facility,
            showlegend=True
        ),
        row=idx, col=1, secondary_y=False
    )
    
    # Add radiation line (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=fac_data['date_hour'],
            y=fac_data['shortwave_radiation'],
            name=f'{facility} Radiation',
            line=dict(color='orange', width=2, dash='dash'),
            legendgroup=facility,
            showlegend=True
        ),
        row=idx, col=1, secondary_y=True
    )
    
    # Update y-axes labels
    fig.update_yaxes(title_text="Energy (MWh)", row=idx, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Radiation (W/m²)", row=idx, col=1, secondary_y=True)

fig.update_xaxes(title_text="Date", row=n_facilities, col=1)
fig.update_layout(
    title=f'Energy (Blue) vs Shortwave Radiation (Orange) - 1 Month ({start_date.date()} to {end_date.date()})',
    height=250*n_facilities,
    width=1200,
    hovermode='x unified'
)

fig.show()

📊 Plotting data from 2025-06-01 to 2025-07-01 (~1 month)
   Total records: 3,600


## 9a. Diagnose Energy-Radiation Mismatch (Detailed Analysis)

Kiểm tra chi tiết các nguyên nhân gây lệch giữa Energy và Radiation:

In [ ]:
print("="*80)
print("ROOT CAUSE ANALYSIS: Energy-Radiation Mismatch")
print("="*80)
print()

# Issue 1: Check for timezone mismatch
print("1️⃣ CHECK TIMEZONE ALIGNMENT")
print("-" * 80)
print("\nEnergy and Radiation should have same date_hour timestamps")
print(f"Energy min date_hour: {energy_df['date_hour'].min()}")
print(f"Energy max date_hour: {energy_df['date_hour'].max()}")
print(f"Weather min date_hour: {weather_df['date_hour'].min()}")
print(f"Weather max date_hour: {weather_df['date_hour'].max()}")

# Check alignment
merged_count = len(df)
energy_count = len(energy_df)
weather_count = len(weather_df)
print(f"\nData alignment:")
print(f"  Energy records: {energy_count}")
print(f"  Weather records: {weather_count}")
print(f"  Merged records: {merged_count}")
print(f"  Match rate: {100*merged_count/max(energy_count, weather_count):.2f}%")

if merged_count < max(energy_count, weather_count):
    print("\n⚠️  WARNING: Some records couldn't be matched!")
    print("   This means some Energy or Radiation data is missing for certain hours")


In [ ]:
print("\n2️⃣ CHECK FOR MISSING/NULL VALUES")
print("-" * 80)

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    energy_nulls = fac_data['energy_mwh'].isna().sum()
    radiation_nulls = fac_data['shortwave_radiation'].isna().sum()
    both_valid = ((~fac_data['energy_mwh'].isna()) & (~fac_data['shortwave_radiation'].isna())).sum()
    
    print(f"\n{facility}:")
    print(f"  Energy records: {len(fac_data)}")
    print(f"  - NULL values: {energy_nulls}")
    print(f"  - Valid values: {len(fac_data) - energy_nulls}")
    print(f"  Radiation records: {len(fac_data)}")
    print(f"  - NULL values: {radiation_nulls}")
    print(f"  - Valid values: {len(fac_data) - radiation_nulls}")
    print(f"  Both valid: {both_valid}")
    
    if energy_nulls > 0 or radiation_nulls > 0:
        print(f"  ⚠️  Missing data detected!")


In [ ]:
print("\n3️⃣ CHECK FOR NIGHT TIME HOURS (Low Radiation with Zero Energy)")
print("-" * 80)
print("\nAt night (shortwave_radiation ≈ 0), energy should also be ≈ 0")
print("If energy is high at night → Data quality issue")
print()

# Define night as radiation < 10 W/m²
night_threshold = 10

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    night_hours = fac_data[fac_data['shortwave_radiation'] < night_threshold]
    
    if len(night_hours) > 0:
        night_energy_mean = night_hours['energy_mwh'].mean()
        night_energy_max = night_hours['energy_mwh'].max()
        night_energy_with_production = (night_hours['energy_mwh'] > 0.1).sum()
        
        print(f"{facility}:")
        print(f"  Night hours (radiation < {night_threshold}): {len(night_hours)}")
        print(f"  Energy at night - mean: {night_energy_mean:.4f}, max: {night_energy_max:.4f}")
        print(f"  Hours with energy > 0.1: {night_energy_with_production} ({100*night_energy_with_production/len(night_hours):.2f}%)")
        
        if night_energy_with_production > len(night_hours) * 0.01:
            print(f"  ⚠️  ANOMALY: {night_energy_with_production} hours have unexpected energy at night!")


In [ ]:
print("\n4️⃣ CHECK FOR TIMEZONE SHIFT IN TIMESTAMPS")
print("-" * 80)
print("\nPossible issue: date_hour might be in different timezones")
print("Let's check if Energy is offset by a fixed number of hours from Radiation\n")

# For each facility, check if there's a consistent time shift
for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility].sort_values('date_hour').copy()
    
    # Get energy and radiation tables separately
    energy_fac = energy_df[energy_df['facility_code'] == facility].sort_values('date_hour')
    weather_fac = weather_df[weather_df['facility_code'] == facility].sort_values('date_hour')
    
    print(f"{facility}:")
    print(f"  Energy hours range: {energy_fac['date_hour'].min()} to {energy_fac['date_hour'].max()}")
    print(f"  Weather hours range: {weather_fac['date_hour'].min()} to {weather_fac['date_hour'].max()}")
    
    # Check if hours are identical
    energy_hours = set(energy_fac['date_hour'].values)
    weather_hours = set(weather_fac['date_hour'].values)
    
    if energy_hours == weather_hours:
        print(f"  ✓ Hour ranges MATCH perfectly")
    else:
        missing_in_energy = weather_hours - energy_hours
        missing_in_weather = energy_hours - weather_hours
        
        if missing_in_energy:
            print(f"  ⚠️  {len(missing_in_energy)} hours in Weather but NOT in Energy")
            if len(missing_in_energy) <= 5:
                print(f"     Missing hours: {sorted(missing_in_energy)}")
        if missing_in_weather:
            print(f"  ⚠️  {len(missing_in_weather)} hours in Energy but NOT in Weather")
            if len(missing_in_weather) <= 5:
                print(f"     Extra hours: {sorted(missing_in_weather)}")


In [ ]:
print("\n5️⃣ SHOW MISMATCHED HOURS SIDE-BY-SIDE")
print("-" * 80)
print("\nFor each facility, show hours where Energy and Radiation don't align:\n")

for facility in sorted(df['facility_code'].unique()):
    energy_fac = energy_df[energy_df['facility_code'] == facility].sort_values('date_hour')
    weather_fac = weather_df[weather_df['facility_code'] == facility].sort_values('date_hour')
    
    energy_hours = set(energy_fac['date_hour'].values)
    weather_hours = set(weather_fac['date_hour'].values)
    
    missing_in_energy = sorted(weather_hours - energy_hours)
    missing_in_weather = sorted(energy_hours - weather_hours)
    
    if missing_in_energy or missing_in_weather:
        print(f"\n{facility}:")
        if missing_in_energy:
            print(f"  Weather has these {len(missing_in_energy)} hours but Energy doesn't:")
            for dt in missing_in_energy[:5]:
                print(f"    - {pd.Timestamp(dt)}")
            if len(missing_in_energy) > 5:
                print(f"    ... and {len(missing_in_energy)-5} more")
        
        if missing_in_weather:
            print(f"  Energy has these {len(missing_in_weather)} hours but Weather doesn't:")
            for dt in missing_in_weather[:5]:
                print(f"    - {pd.Timestamp(dt)}")
            if len(missing_in_weather) > 5:
                print(f"    ... and {len(missing_in_weather)-5} more")

In [ ]:
print("\n6️⃣ DETAILED VIEW OF TIMESTAMP COLUMNS IN SOURCE TABLES")
print("-" * 80)
print("\nLet's check the raw source data to understand timestamp structure:\n")

# Query Bronze tables directly to see raw timestamps
query_energy_raw = """
SELECT 
    interval_ts,
    DATE_TRUNC('hour', interval_ts) as interval_ts_hour,
    facility_code,
    metric,
    value
FROM iceberg.bronze.raw_facility_timeseries
WHERE facility_code = 'NYNGAN'
LIMIT 10
"""

query_weather_raw = """
SELECT 
    weather_timestamp,
    DATE_TRUNC('hour', weather_timestamp) as weather_ts_hour,
    facility_code,
    shortwave_radiation
FROM iceberg.bronze.raw_facility_weather
WHERE facility_code = 'NYNGAN'
LIMIT 10
"""

print("Sample Energy (Bronze):")
energy_raw = pd.read_sql(query_energy_raw, conn)
print(energy_raw.to_string())

print("\n\nSample Weather (Bronze):")
weather_raw = pd.read_sql(query_weather_raw, conn)
print(weather_raw.to_string())

## 10. Correlation Heatmap

In [ ]:
# Calculate correlation for each facility separately
fig, axes = plt.subplots(1, len(facilities), figsize=(15, 4))

for idx, facility in enumerate(facilities):
    fac_data = df[df['facility_code'] == facility][[
        'energy_mwh', 'shortwave_radiation', 
        'direct_radiation', 'diffuse_radiation', 'direct_normal_irradiance'
    ]].dropna()
    
    corr_matrix = fac_data.corr()
    
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        center=0,
        vmin=-1,
        vmax=1,
        ax=axes[idx],
        cbar=False
    )
    axes[idx].set_title(f'{facility}', fontsize=12, fontweight='bold')

plt.suptitle('Correlation Matrix: Energy vs Radiation Metrics', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()